In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from torchvision import models,transforms
from torch.utils.data import Dataset, DataLoader
import os
import cv2
import glob
from PIL import Image
import PIL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mnist_train = torchvision.datasets.MNIST('/content/drive/My Drive/Dataset/',train=True,transform=transforms.ToTensor())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
img,label = mnist_train[0]

In [ ]:
img=img.view(28,28)

In [ ]:
def tensor_to_img(img):
  img=img*255
  img=np.array(img,dtype=np.uint8)
  return PIL.Image.fromarray(img)
def means(i):
  ten,classe=mnist_train[i]
  ten=ten.view(28,28)
  return (ten,classe)

In [ ]:
image = tensor_to_img(img)

In [ ]:
display(image)

In [ ]:
img2,cll = mnist_train[4]
img2=img2.view(28,28)
display(tensor_to_img(img2))
print(img2.shape)
print(cll)

torch.Size([28, 28])
9


In [ ]:
ten_ten=torch.empty(60000,28,28)
ten_classe=torch.empty(60000,1)
for i in range(60000):
  ten_ten[i],ten_classe[i]=means(i)



In [ ]:
ten_array=ten_ten.numpy()

In [ ]:
mean=np.mean(ten_array)
std=np.std(ten_array)

In [ ]:
print(mean)
print(std)
ten_classe

0.13066062
0.30810776


tensor([[5.],
        [0.],
        [4.],
        ...,
        [5.],
        [6.],
        [8.]])

In [ ]:
tenss=means(3)

In [ ]:
class reshape:
  def __call__(self,sample):
    inputs=sample
    return inputs.view(28,28)

In [ ]:
data_transforms=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((mean),(std))
])

In [ ]:
mnist_train_fin = torchvision.datasets.MNIST('/content/drive/My Drive/Dataset/',train=True,transform=data_transforms)

In [ ]:
tensor,classuski=mnist_train_fin[i]

In [ ]:
print(type(classuski))

<class 'int'>


In [ ]:
tensor.shape

torch.Size([1, 28, 28])

In [ ]:
train_loader=DataLoader(dataset=mnist_train_fin,batch_size=128,shuffle=True)    

In [ ]:
train_loader

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1=nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3)
    self.fc1=nn.Linear(1936,1000)
    self.fc2=nn.Linear(1000,10)
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=F.relu(self.conv2(x))
    x=x.view(-1,1936)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return x

In [ ]:
model=ConvNet().to(device)

In [ ]:
learning_rate=0.01
criteria=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
decay_scheduler=lr_scheduler.StepLR(optimizer,step_size=40,gamma=0.1)

In [ ]:
num_epochs=40
def train(model,criteria,optimizer,num_epochs,scheduler):
    for epoch in range(num_epochs):
        print(f'Epoch-{epoch}')
        print('-'*10)
        num=0
        epoch_loss=0
        n_samples=0
        correct=0
        for j,(inputs,labels) in enumerate(train_loader):
            inputs=inputs.to(device)
            labels=labels.to(device)
            
            output=model(inputs)
            
            _,preds=torch.max(output,1)
            correct+=(preds == labels).sum().item()
            n_samples += labels.size(0)

            loss=criteria(output,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss+=loss.item()
            
            if(j%60==0):
                print(f'Loss-{loss.item()}')
            num=j
        
        print(f'Epoch Loss- {epoch_loss/num} Epoch Accuracy-{(correct/n_samples)*100}' )  
        scheduler.step()
    return model

In [ ]:
model_trained=train(model,criteria,optimizer,num_epochs,decay_scheduler)

Epoch-0
----------
Loss-2.310866355895996


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Loss-1.0843969583511353
Loss-0.446454793214798
Loss-0.38808271288871765
Loss-0.32258930802345276
Loss-0.309639036655426
Loss-0.32634487748146057
Loss-0.25807738304138184
Epoch Loss- 0.6061157671750611 Epoch Accuracy-83.09
Epoch-1
----------
Loss-0.19431036710739136
Loss-0.32078316807746887
Loss-0.21052633225917816
Loss-0.2246464341878891
Loss-0.23378416895866394
Loss-0.2540442943572998
Loss-0.30494093894958496
Loss-0.20685702562332153
Epoch Loss- 0.23331279788389164 Epoch Accuracy-93.07666666666667
Epoch-2
----------
Loss-0.25292009115219116
Loss-0.16885247826576233
Loss-0.22096681594848633
Loss-0.28534162044525146
Loss-0.1537814438343048
Loss-0.13143512606620789
Loss-0.1297464519739151
Loss-0.1928417682647705
Epoch Loss- 0.17126181088069564 Epoch Accuracy-94.99333333333333
Epoch-3
----------
Loss-0.11291778832674026
Loss-0.09560094028711319
Loss-0.21943381428718567
Loss-0.12026049196720123
Loss-0.1472267359495163
Loss-0.08861462771892548
Loss-0.1495811939239502
Loss-0.171619713306427


In [ ]:
mnist_test = torchvision.datasets.MNIST('/content/drive/My Drive/Dataset/',download=True,train=False,transform=transforms.ToTensor())

In [ ]:
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Root location: /content/drive/My Drive/Dataset/
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
mnist_test_fin = torchvision.datasets.MNIST('/content/drive/My Drive/Dataset/',train=False,transform=data_transforms)

In [ ]:
mnist_test_fin

Dataset MNIST
    Number of datapoints: 10000
    Root location: /content/drive/My Drive/Dataset/
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.13066062331199646, std=0.30810776352882385)
           )

In [ ]:
test_loader=DataLoader(dataset=mnist_test_fin,batch_size=128)

In [ ]:
test_loader

In [ ]:
test_correct=0
test_n_samples=0
with torch.no_grad():
  for (inps,labs) in test_loader:
    inps=inps.to(device)
    labs=labs.to(device) 
  
    outs=model(inps)
    _,predss=torch.max(outs,1)
    
    test_correct += (predss==labs).sum().item()    
    test_n_samples += labs.size(0)





In [ ]:
print(100*(test_correct/test_n_samples))

98.33
